In [2]:
import matplotlib
import operator
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import joblib

# Set 
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.size'] = 10

# import custom functions
from functions_statistical_performance import *

In [3]:
# True to exactly reproduce the results presented in the paper (same classifiers)
reproduceflag=False

In [4]:
# function to predict over a generic db when you already have the classifiers
# it provides the average FPro 
def ClassifyDB(db, model_per_fold,nut_sel):
    dbsel=db.loc[:,nut_sel]
    Xnut=dbsel.values    
    
    indfold=0
    for model in model_per_fold:
        indfold+=1
        y_pred = model.predict(Xnut)
        y_probs= model.predict_proba(Xnut)
        db['classf'+str(indfold)]=y_pred
        db['p1f'+str(indfold)]=y_probs[:,0]
        db['p2f'+str(indfold)]=y_probs[:,1]
        db['p3f'+str(indfold)]=y_probs[:,2]
        db['p4f'+str(indfold)]=y_probs[:,3]
        db['FProf'+str(indfold)]=(1-db['p1f'+str(indfold)]+db['p4f'+str(indfold)])/2
     
    for p in range(1,5):
        db['p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].mean(axis=1)
        db['std_p'+str(p)]=db.loc[:, ['p'+str(p)+'f1', 'p'+str(p)+'f2', 'p'+str(p)+'f3','p'+str(p)+'f4', 'p'+str(p)+'f5' ]].std(axis=1)


    db['FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5' ]].mean(axis=1)
    db['std_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].std(axis=1)
    db['min_FPro']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].min(axis=1)


    db['max_p']=db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)
    db['class']=[int(s[1])-1 for s in db.loc[:, ['p1', 'p2', 'p3','p4' ]].idxmax(axis=1)]
    db['min_in_which_fold']=db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)
    db['min_fold_id']=[int(s[-1]) for s in db.loc[:, ['FProf1', 'FProf2', 'FProf3','FProf4', 'FProf5']].idxmin(axis=1)]
    db['min_class']=[db['classf'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    
    for ind in range(1,5):
        db['p'+str(ind)+'_minFPro']=[db['p'+str(ind)+'f'+str(db['min_fold_id'].iloc[n])].iloc[n] for n in range(db.shape[0])]
    

    return db

# Load 2009-2010 Data

In [5]:
# log-transformed nutrient content per 100 grams
RFFNDDS=pd.read_csv('./input_data/FNDDS_2009-10_Training_Data.csv')
RFFNDDS

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,Total flavones,Total flavonols,Total isoflavones,Food_code,Main_food_description,catnumb,catname,novaclass,macroclass,pythonlabel
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,-20.0,-20.0,-20.0,2047,"Salt, table",0,addition1516,2,Other,1
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,-20.0,-20.0,-20.0,11000000,"Milk, human",9602,Human milk,1,Milk,0
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,-20.0,-20.0,-20.0,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,Milk,0
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,-20.0,-20.0,-20.0,11111000,"Milk, cow's, fluid, whole",1002,"Milk, whole",1,Milk,0
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,-20.0,-20.0,-20.0,11111100,"Milk, cow's, fluid, whole, low-sodium",1002,"Milk, whole",1,Milk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94100200,"Water, bottled, sweetened, with low or no calo...",7802,Flavored or carbonated water,4,Other,3
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94100300,"Water, fruit flavored, sweetened, with high fr...",7802,Flavored or carbonated water,4,Other,3
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,-20.0,-20.0,-20.0,94210100,Propel Water,7804,Enhanced or fortified water,4,Other,3
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,-20.0,-20.0,-20.0,94210200,Glaceau Water,7804,Enhanced or fortified water,4,Other,3


In [6]:
# total nutrient list
nl=list(RFFNDDS.columns)
nl=nl[0:-7]
len(nl)

99

# Whole Nutrient Panel Analysis

In [7]:
if reproduceflag:
    model_per_foldWNP=joblib.load("./paper_classifiers_outcomes/FNDDS0910_99Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDS[RFFNDDS.pythonlabel!=-1]
    X=data2train.loc[:, 'Protein': 'Total isoflavones']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUCWNP, performancesAUPWNP, splitsWNP, model_per_foldWNP)=AUCAUPkfold(X,y,smoteflag)


AUC: average over the folds
[0.97974486 0.98730014 0.97054765 0.98037994]
AUC: std over the folds
[0.00357327 0.00791888 0.00492999 0.00493372]
AUP: average over the folds
[0.88766196 0.79158659 0.88714138 0.99165199]
AUP: std over the folds
[0.02033712 0.08155604 0.00505044 0.00242217]
Balanced Accuracy: average over the folds
0.7815171752831354
Balanced Accuracy: std over the folds
0.04270354220663592


In [8]:
# summary database
df=RFFNDDS.copy()
df=ClassifyDB(df,model_per_foldWNP, nl)
df.to_csv('./output_data/FNDDS_2009-10_99_Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.625000,p2,1,FProf3,3,1,0.015000,0.680000,0.040,0.265000
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.295000,p1,0,FProf5,5,0,0.675000,0.010000,0.050,0.265000
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.020000,p1,0,FProf4,4,0,0.975000,0.000000,0.010,0.015000
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.000000,p1,0,FProf4,4,0,1.000000,0.000000,0.000,0.000000
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.120417,p1,0,FProf5,5,0,0.862083,0.010000,0.025,0.102917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.668750,p4,3,FProf4,4,3,0.325000,0.012500,0.000,0.662500
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.858750,p4,3,FProf3,3,3,0.105000,0.027500,0.045,0.822500
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.962500,p4,3,FProf3,3,3,0.030000,0.000000,0.015,0.955000
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.769583,p4,3,FProf3,3,3,0.140000,0.105833,0.075,0.679167


# Standard Release FNDDS (for releases without flavonoids)

In [9]:
RFFNDDSSR=RFFNDDS.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
StandardNutPanel=list(RFFNDDSSR.columns)
RFFNDDSSR['pythonlabel']=RFFNDDS['pythonlabel'].copy()
print('Number of Nutrients in the Standard Nutrient Panel')
print(len(StandardNutPanel))

Number of Nutrients in the Standard Nutrient Panel
62


In [11]:
if reproduceflag:
    model_per_fold62nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_62Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSSR[RFFNDDSSR.pythonlabel!=-1]
    data2train['pythonlabel'] = np.random.permutation(data2train['pythonlabel'].values)
    X=data2train.loc[:, 'Protein': 'Fatty acids, total polyunsaturated']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUC62nut, performancesAUP62nut, splits62nut, model_per_fold62nut)=AUCAUPkfold(X,y,smoteflag)

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


AUC: average over the folds
[0.47988169 0.54715536 0.53455942 0.50442664]
AUC: std over the folds
[0.02850633 0.07994514 0.01557605 0.01391974]
AUP: average over the folds
[0.11452427 0.02835294 0.17579022 0.72224894]
AUP: std over the folds
[0.0071046  0.00578507 0.00868933 0.01525558]
Balanced Accuracy: average over the folds
0.2491558050436476
Balanced Accuracy: std over the folds
0.0015625518453723716


In [11]:
# summary database
df=RFFNDDSSR.copy()
df=ClassifyDB(df,model_per_fold62nut,StandardNutPanel)
df.to_csv('./output_data/FNDDS_2009-10_62Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,Alcohol,Water,Caffeine,Theobromine,"Sugars, total","Fiber, total dietary",Calcium,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.0,-1.609438,-20.0,-20.0,-20.000000,-20.0,-3.729701,...,0.704899,p4,3,FProf3,3,3,0.201476,0.030000,0.157250,0.611274
1,0.029559,1.477049,1.930071,-20.0,4.471639,-20.0,-20.0,1.930071,-20.0,-3.442019,...,0.566875,p3,2,FProf4,4,2,0.082500,0.010000,0.691250,0.216250
2,1.187843,0.647103,1.578979,-20.0,4.491441,-20.0,-20.0,1.625311,-20.0,-2.128632,...,0.640000,p3,2,FProf2,2,2,0.029167,0.001667,0.660000,0.309167
3,1.147402,1.178655,1.568616,-20.0,4.478813,-20.0,-20.0,1.619388,-20.0,-2.180367,...,0.871667,p4,3,FProf5,5,3,0.010000,0.000000,0.236667,0.753333
4,1.131402,1.241269,1.495149,-20.0,4.479607,-20.0,-20.0,1.495149,-20.0,-2.292635,...,0.544583,p3,2,FProf1,1,2,0.085833,0.019167,0.720000,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.0,4.603669,-20.0,-20.0,-20.000000,-20.0,-6.907755,...,0.651242,p4,3,FProf2,2,3,0.253688,0.105750,0.084389,0.556173
7250,-20.000000,-20.000000,1.504077,-20.0,4.547965,-20.0,-20.0,1.504077,-20.0,-4.074542,...,0.792625,p4,3,FProf5,5,3,0.134250,0.010000,0.136250,0.719500
7251,-20.000000,-20.000000,0.198851,-20.0,4.592895,-20.0,-20.0,0.198851,-20.0,-6.907755,...,0.729583,p4,3,FProf3,3,3,0.138333,0.015000,0.249167,0.597500
7252,-20.000000,-20.000000,1.702928,-20.0,4.547965,-20.0,-20.0,1.702928,-20.0,-4.074542,...,0.889435,p4,3,FProf2,2,3,0.058393,0.005000,0.099345,0.837262


# Classify FNDDS 2015-2016 with 62 Nutrients

In [12]:
# log-transformed nutrient content per 100 grams
RFFNDDS20152016=pd.read_csv('./input_data/FNDDS_2015-16.csv')
df=ClassifyDB(RFFNDDS20152016,model_per_fold62nut,StandardNutPanel)
df.to_csv("./output_data/FNDDS_2015-16_62Nutrients_ensemble_5folds.csv")
df

C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Food code,Main food description,year,cat_digit_1,cat_digit_2,cat_digit_3,cat_digit_4,cat_digit_5,WWEIA Category number,WWEIA Category description,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,11000000,"Milk, human",2015,Milk and Milk Products,Milks and milk drinks,"Milk, human",NaN,NaN,9602,Human milk,...,0.566875,p3,2,FProf4,4,2,0.082500,0.010000,0.691250,0.216250
1,11100000,"Milk, NFS",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1004,"Milk, reduced fat",...,0.731667,p4,3,FProf5,5,3,0.035833,0.000000,0.465000,0.499167
2,11111000,"Milk, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.871667,p4,3,FProf5,5,3,0.010000,0.000000,0.236667,0.753333
3,11111100,"Milk, low sodium, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.544583,p3,2,FProf1,1,2,0.085833,0.019167,0.720000,0.175000
4,11111150,"Milk, calcium fortified, whole",2015,Milk and Milk Products,Milks and milk drinks,"Milk, fluid (regular; filled; buttermilk; and ...",NaN,NaN,1002,"Milk, whole",...,0.871667,p4,3,FProf5,5,3,0.010000,0.000000,0.236667,0.753333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,99991400,Cheese as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.708000,p4,3,FProf5,5,3,0.232000,0.005000,0.115000,0.648000
8686,99995000,Breading or batter as ingredient in food,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.759375,p4,3,FProf1,1,3,0.098333,0.025000,0.259583,0.617083
8687,99995130,Wheat bread as ingredient in sandwiches,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.837500,p4,3,FProf4,4,3,0.115000,0.030000,0.065000,0.790000
8688,99998130,Sauce as ingredient in hamburgers,2015,"Sugars, Sweets, and Beverages",NaN,NaN,NaN,NaN,9999,Not included in a food category,...,0.762500,p4,3,FProf4,4,3,0.145000,0.005000,0.180000,0.670000


# Nutrition Facts Analysis

In [13]:
nutrition_facts=['Protein','Total Fat','Carbohydrate','Sugars, total','Fiber, total dietary','Calcium','Iron', 'Sodium','Retinol','Carotene, beta',
 'Carotene, alpha','Cryptoxanthin, beta','Vitamin C','Cholesterol', 'Fatty acids, total saturated'];
VitaminA=['Retinol','Carotene, beta','Carotene, alpha','Cryptoxanthin, beta']
RFFNDDSNF=RFFNDDS[nutrition_facts]
RFFNDDSNF['Total Vitamin A']=RFFNDDS[VitaminA].apply(np.exp).sum(axis=1).apply(np.log)
RFFNDDSNF=RFFNDDSNF.drop(VitaminA, axis=1)
nutrition_facts=list(RFFNDDSNF.columns)
print('Number of Nutrients in the Nutrition Facts Panel')
print(len(nutrition_facts))
RFFNDDSNF['pythonlabel']=RFFNDDS['pythonlabel'].copy()

Number of Nutrients in the Nutrition Facts Panel
12


C:\Users\harel\PycharmProjects\MLFoodProcessing-auto\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
if reproduceflag:
    model_per_fold12nut=joblib.load("./paper_classifiers_outcomes/FNDDS0910_12Nutrients_5folds_SMOTE.pkl")
else:
    data2train=RFFNDDSNF[RFFNDDSNF.pythonlabel!=-1]
    X=data2train.loc[:, 'Protein': 'Total Vitamin A']
    y=data2train['pythonlabel']
    smoteflag=False
    (performancesAUC12nut, performancesAUP12nut, splits12nut, model_per_fold12nut)=AUCAUPkfold(X,y,smoteflag)

AUC: average over the folds
[0.98275523 0.98598562 0.96764752 0.97917586]
AUC: std over the folds
[0.00379044 0.00730989 0.00590724 0.00470058]
AUP: average over the folds
[0.89809058 0.79702508 0.86621887 0.9911194 ]
AUP: std over the folds
[0.02796458 0.06116615 0.01856124 0.00202637]
Balanced Accuracy: average over the folds
0.7870607338343361
Balanced Accuracy: std over the folds
0.020028650488407476


In [15]:
# summary database
df=RFFNDDSNF.copy()
df=ClassifyDB(df,model_per_fold12nut,nutrition_facts)
df.to_csv('./output_data/FNDDS_2009-10_12Nutrients_ensemble_5folds.csv')
df

,Protein,Total Fat,Carbohydrate,"Sugars, total","Fiber, total dietary",Calcium,Iron,Sodium,Vitamin C,Cholesterol,...,min_FPro,max_p,class,min_in_which_fold,min_fold_id,min_class,p1_minFPro,p2_minFPro,p3_minFPro,p4_minFPro
0,-20.000000,-20.000000,-20.000000,-20.000000,-20.0,-3.729701,-8.016418,3.657337,-20.000000,-20.000000,...,0.584167,p2,1,FProf1,1,1,0.080000,0.635,0.036667,0.248333
1,0.029559,1.477049,1.930071,1.930071,-20.0,-3.442019,-10.414313,-4.074542,-5.298317,-4.268698,...,0.325000,p1,0,FProf2,2,0,0.660000,0.005,0.025000,0.310000
2,1.187843,0.647103,1.578979,1.625311,-20.0,-2.128632,-10.414313,-3.123566,-9.210340,-4.961845,...,0.005000,p1,0,FProf5,5,0,0.995000,0.000,0.000000,0.005000
3,1.147402,1.178655,1.568616,1.619388,-20.0,-2.180367,-10.414313,-3.146555,-20.000000,-4.605170,...,0.006667,p1,0,FProf5,5,0,0.993333,0.000,0.000000,0.006667
4,1.131402,1.241269,1.495149,1.495149,-20.0,-2.292635,-9.903488,-5.809143,-7.013116,-4.268698,...,0.135000,p1,0,FProf1,1,0,0.840000,0.030,0.020000,0.110000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,-20.000000,-20.000000,-1.897120,-20.000000,-20.0,-6.907755,-20.000000,-5.809143,-20.000000,-20.000000,...,0.377500,p4,3,FProf3,3,0,0.620000,0.005,0.000000,0.375000
7250,-20.000000,-20.000000,1.504077,1.504077,-20.0,-4.074542,-20.000000,-4.828314,-4.366153,-20.000000,...,0.892500,p4,3,FProf3,3,3,0.095000,0.000,0.025000,0.880000
7251,-20.000000,-20.000000,0.198851,0.198851,-20.0,-6.907755,-10.414313,-4.342806,-4.733004,-20.000000,...,0.982500,p4,3,FProf1,1,3,0.010000,0.000,0.015000,0.975000
7252,-20.000000,-20.000000,1.702928,1.702928,-20.0,-4.074542,-20.000000,-20.000000,-4.366153,-20.000000,...,0.822500,p4,3,FProf1,1,3,0.115000,0.045,0.080000,0.760000
